# RBM(Restricted Boltzmann Machine)

## Importing the libraries

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.optim as  optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.parallel

## Importing the dataset

In [26]:
movies = pd.read_csv("movies.dat",sep="::",header=None,engine='python',encoding="latin-1")
users = pd.read_csv("users.dat",sep="::",header=None,engine='python',encoding="latin-1")
ratings = pd.read_csv("ratings.dat",sep="::",header=None,engine='python',encoding="latin-1")


## Preparing the training set and the test set

In [27]:
training_set = pd.read_csv("u1.base",delimiter='\t')
training_set = np.array(training_set,dtype='int')
test_set = pd.read_csv("u1.test",delimiter='\t')
test_set = np.array(test_set,dtype='int')

## Getting the number of users and movies

In [28]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
print(nb_users)
print(nb_movies)
 # ene bol yuruusu l split ni random shaatsan bsn bolohoor , hamgiin ih id-tai user ni test_set nd baigaayu? training_set nd baigaayu gedgiig uurur medeh argagui shaagaad baisiin.

943
1682


## Converting the data into an array with users in lines and movies in columns

In [29]:
def convert(data) :
    new_data = list()
    for id_users in range(1,nb_users+1) :
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)
# ene bol list of lists bolj baigaaz. dotor taliiinh ni bol 1682 elemennttei, ugsun onoo ni baina.
# gadna taliinh ni 943n user-iin tootoi shit.

## Converting the data into Torch tensors

In [30]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [31]:
training_set[training_set==0] = -1
training_set[training_set==1] = 0
training_set[training_set==2] = 0
training_set[training_set>=3] = 1
test_set[test_set==0] = -1
test_set[test_set==1] = 0
test_set[test_set==3] = 0
test_set[test_set>=3] = 1

## Creating the arhitecture of the Neural Network

In [32]:
class RBM() :
    def __init__(self,nv,nh) :  #nv - number of visible nodes,nh-number of hidden nodes
        self.W = torch.randn(nv,nh)  # weights matrix.Connects visible and hidden units. # ene shit ni bol normal distribution hiigeed ugchij baigaa l gejiin.
        self.a = torch.randn(1,nh)   # bias for hidden shits  2D dimension-toi bolgohiin tuld l 1iig nemj ugch baigaa genee.
        self.b = torch.randn(1,nv)   # bias for visible nodes.

    def sample_h(self,x) :  # Sample hidden units given visible units (sample_h).
        wx = torch.mm(x,self.W)  # computes the dot product of x , and transpose of the weight matrix
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)  # p_h_given_v: Probability distribution over hidden neurons.
        return p_h_given_v , torch.bernoulli(p_h_given_v)  # Samples binary hidden states (0 or 1) based on the probability.

    def sample_v(self,y) :  # Reconstruct visible units and resample hidden units (sample_v).
        wy = torch.mm(y,self.W.t())
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)  # Applies the sigmoid function to calculate the activation probability for visible neurons given the hidden neurons.
        return p_v_given_h , torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)   # visible bias
        self.a += torch.sum((ph0 - phk), 0) # hidden bias

nv = len(training_set[0])  # Number of visible units (features in input data)
nh = 100   # Number of hidden units
batch_size = 100
rbm = RBM(nv,nh)

# Inputs to the train method.
- v0: Initial visible state (input data).
- vk: Reconstructed visible state after Gibbs sampling.
- ph0: Probability of hidden units given initial visible state.
- phk: Probability of hidden units given reconstructed visible state.
# Weight update
- Positive Phase: torch.mm(v0.t(), ph0).t() (correlations from the data distribution).
- Negative Phase: torch.mm(vk.t(), phk).t() (correlations from the model distribution).
# Bias Update
- Visible Bias (b): self.b += torch.sum((v0 - vk), 0)
- Hidden Bias (a): self.a += torch.sum((ph0 - phk), 0)


## Training the RBM

In [33]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]  # selects the batch size of 100 users.
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10): # Gibbs Sampling: k-step Contrastive Divergence  .. ene deer yamr ch update baihgu. zugeerl vk, hk 2oo negneer ni sample hiigeed yavaad baigaa. tegj baigaad,ali her yalgaatai baina gedgeer n update hiine. ugaasa neg l udaa sample hiichih ym bol, heterhii adilhan bolood,oilgtoi sain train hiigdej chadku bilee.
            _,hk = rbm.sample_h(vk)  # returns first sampled hidden nodes.
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # vk[v0 < 0] = v0[v0 < 0] ensures that missing data (e.g., unrated movies) remains unchanged.
        phk,_ = rbm.sample_h(vk) # calculates the probabilities of hidden units based on the reconstructed visible layer (vk).
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3609)
epoch: 2 loss: tensor(0.2547)
epoch: 3 loss: tensor(0.2478)
epoch: 4 loss: tensor(0.2491)
epoch: 5 loss: tensor(0.2445)
epoch: 6 loss: tensor(0.2479)
epoch: 7 loss: tensor(0.2480)
epoch: 8 loss: tensor(0.2489)
epoch: 9 loss: tensor(0.2450)
epoch: 10 loss: tensor(0.2468)


## Testing the RBM

In [35]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.4438)
